## Deadline: 5 June 2023

### 1. FM-index

Implement FM-index data structure and string search using it. Provide tests.

https://www.youtube.com/watch?v=4n7NPk5lwbI

3 points

In [ ]:
class FMIndex:
    def __init__(self, text):
        self.text = text + '$'
        self.bwt, self.sa = self._construct_bwt_and_sa()
        self.counts, self.first_occurrence = self._preprocess_bwt()

    def _construct_bwt_and_sa(self):
        suffixes = [self.text[i:] + self.text[:i] for i in range(len(self.text))]
        suffixes.sort()
        bwt = ''.join([suffix[-1] for suffix in suffixes])
        sa = [len(self.text) - suffix.index('$') - 1 for suffix in suffixes]
        return bwt, sa

    def _preprocess_bwt(self):
        counts = {}
        first_occurrence = {}
        for char in self.bwt:
            counts[char] = counts.get(char, 0) + 1
        sorted_chars = sorted(counts.keys())
        total = 0
        for char in sorted_chars:
            first_occurrence[char] = total
            total += counts[char]
        return counts, first_occurrence

    def _get_occurrences(self, char):
        start = self.first_occurrence[char]
        if char == '$':
            end = len(self.bwt)
        else:
            end = self.first_occurrence[char] + self.counts[char]
        return start, end

    def _backward_search(self, pattern):
        top, bottom = 0, len(self.bwt)
        for i in range(len(pattern) - 1, -1, -1):
            char = pattern[i]
            start, end = self._get_occurrences(char)
            top = start + self._count_occurrences(char, top - 1)
            bottom = start + self._count_occurrences(char, bottom) - 1
            if top > bottom:
                return []
        return self.sa[top:bottom+1]

    def _count_occurrences(self, char, index):
        count = 0
        for i in range(index):
            if self.bwt[i] == char:
                count += 1
        return count

    def search(self, pattern):
        occurrences = self._backward_search(pattern)
        return occurrences


In [ ]:
# Example usage and tests
text = "ACGTAGTACGTACGT"
fm_index = FMIndex(text)

# Test search
pattern = "ACGT"
occurrences = fm_index.search(pattern)
print(f"Occurrences of '{pattern}': {occurrences}")  

pattern = "GATTACA"
occurrences = fm_index.search(pattern)
print(f"Occurrences of '{pattern}': {occurrences}")  

pattern = "ATA"
occurrences = fm_index.search(pattern)
print(f"Occurrences of '{pattern}': {occurrences}")  

pattern = "T"
occurrences = fm_index.search(pattern)
print(f"Occurrences of '{pattern}': {occurrences}")  

pattern = "AAG"
occurrences = fm_index.search(pattern)
print(f"Occurrences of '{pattern}': {occurrences}")  

Occurrences of 'ACGT': [11]
Occurrences of 'GATTACA': []
Occurrences of 'ATA': []
Occurrences of 'T': [14, 10, 6, 3]
Occurrences of 'AAG': []


### 2. Needleman–Wunsch

Implement Needleman–Wunsch algorithm. Provide tests.

2 points

In [ ]:
class NeedlemanWunsch:
    def __init__(self, match_score=1, mismatch_penalty=-1, gap_penalty=-1):
        self.match_score = match_score
        self.mismatch_penalty = mismatch_penalty
        self.gap_penalty = gap_penalty

    def align(self, seq1, seq2):
        # Initialize the score matrix and traceback matrix
        n = len(seq1)
        m = len(seq2)
        score = [[0] * (m + 1) for _ in range(n + 1)]
        traceback = [[None] * (m + 1) for _ in range(n + 1)]

        # Initialize the first row and column of the score matrix
        for i in range(1, n + 1):
            score[i][0] = self.gap_penalty * i
            traceback[i][0] = 'U'
        for j in range(1, m + 1):
            score[0][j] = self.gap_penalty * j
            traceback[0][j] = 'L'

        # Fill in the score matrix and traceback matrix
        for i in range(1, n + 1):
            for j in range(1, m + 1):
                match = score[i - 1][j - 1] + (self.match_score if seq1[i - 1] == seq2[j - 1] else self.mismatch_penalty)
                delete = score[i - 1][j] + self.gap_penalty
                insert = score[i][j - 1] + self.gap_penalty
                score[i][j] = max(match, delete, insert)
                if score[i][j] == match:
                    traceback[i][j] = 'D'
                elif score[i][j] == delete:
                    traceback[i][j] = 'U'
                else:
                    traceback[i][j] = 'L'

        # Traceback to find the optimal alignment
        aligned_seq1 = ''
        aligned_seq2 = ''
        i, j = n, m
        while i > 0 and j > 0:
            if traceback[i][j] == 'D':
                aligned_seq1 = seq1[i - 1] + aligned_seq1
                aligned_seq2 = seq2[j - 1] + aligned_seq2
                i -= 1
                j -= 1
            elif traceback[i][j] == 'U':
                aligned_seq1 = seq1[i - 1] + aligned_seq1
                aligned_seq2 = '-' + aligned_seq2
                i -= 1
            else:
                aligned_seq1 = '-' + aligned_seq1
                aligned_seq2 = seq2[j - 1] + aligned_seq2
                j -= 1

        while i > 0:
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = '-' + aligned_seq2
            i -= 1

        while j > 0:
            aligned_seq1 = '-' + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            j -= 1

        return aligned_seq1, aligned_seq2

In [ ]:
seq1 = "ACGTACGT"
seq2 = "AGTACG"
nw = NeedlemanWunsch()
aligned_seq1, aligned_seq2 = nw.align(seq1, seq2)
print(f"Aligned Seq1: {aligned_seq1}")
print(f"Aligned Seq2: {aligned_seq2}")

Aligned Seq1: ACGTACGT
Aligned Seq2: A-GTACG-


### 3*.  (Optional)  Greedy clustring

Implement simple clustering procedure for nucleotide sequences described in 2.2 Cd-hit and cd-hit-est clustering section (https://doi.org/10.1093/bioinformatics/btl158). You can omit short word filtering, use the global alignment implementation from the second task or biopython global or local alignment https://biopython.org/docs/1.75/api/Bio.pairwise2.html, and implement a custom score threshold calculation based on length of the representative cluster sequence (the first sequence added to cluster).

4 points

In [ ]:
! pip install Bio

In [ ]:
from Bio import pairwise2

   Start with an empty list of clusters.
  For each sequence in the input dataset:

  a. Compare it with each representative sequence of the existing clusters.

   b. If the alignment score (based on global alignment) is above a threshold 
    calculated from the length of the representative sequence, add the sequence to that cluster.

  c. If no cluster is found or the alignment score is below the threshold, create a new cluster with the sequence as the representative.
    Return the list of clusters.

In [ ]:
class CdHit:
    def __init__(self, threshold=0.9):
        self.threshold = threshold
        self.clusters = []

    def add_sequence(self, sequence):
        # If no clusters exist, create a new cluster with the sequence as the representative
        if not self.clusters:
            self.clusters.append([sequence])
            return

        # Calculate the alignment threshold based on the length of the representative sequence
        rep_length = len(self.clusters[0][0])
        alignment_threshold = self.threshold * rep_length

        # Compare the sequence with the representative sequences of existing clusters
        for cluster in self.clusters:
            rep_sequence = cluster[0]

            # Perform global alignment between the sequence and the representative sequence
            alignments = pairwise2.align.globalxx(rep_sequence, sequence)
            alignment_score = alignments[0].score

            # If the alignment score is above the threshold, add the sequence to the cluster
            if alignment_score >= alignment_threshold:
                cluster.append(sequence)
                return

        # If no suitable cluster is found, create a new cluster with the sequence as the representative
        self.clusters.append([sequence])

    def cluster_sequences(self, sequences):
        for sequence in sequences:
            self.add_sequence(sequence)
        return self.clusters


# Example usage and test
sequences = [
    "ATCGGATCG",
    "ATCGGATCC",
    "GTCGGAACC",
    "ATCGGATCGA",
    "ATCGGATCCA",
    "GTCGGAACCA"
]

cd_hit = CdHit(threshold=0.9)
clusters = cd_hit.cluster_sequences(sequences)

# Print the clusters
for i, cluster in enumerate(clusters):
    print(f"Cluster {i+1}:")
    for sequence in cluster:
        print(sequence)
    print()


Cluster 1:
ATCGGATCG
ATCGGATCGA

Cluster 2:
ATCGGATCC
ATCGGATCCA

Cluster 3:
GTCGGAACC
GTCGGAACCA

